In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.tools

In [26]:
def linear_regression(data, minimal=False, custom=None):

    #Feature Engineering Function
    def feature_engineering(df):
        df = df.copy()
        df = pd.get_dummies(df, columns=["Region"], drop_first=True, prefix="region", dtype=int)        #One Hot Encoding the region column
        df["GDP_per_capita"] = df["GDP_per_capita"] / 1000                                              #Convert GDP into easier to use units
        df = sm.add_constant(df)
        return df
    
    #Reading the data
    df = pd.read_csv(data)

    #Setting up variables and target
    fcols = list(df.columns)
    fcols.remove("Life_expectancy")
    fcols.remove("Country")
    X = df[fcols]
    y = df.Life_expectancy

    #Train-test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

    #Applying Feature Engineering Function
    X_train_fe = feature_engineering(X_train)

    #Specifying columns to use in case minimal version is used
    if custom:
        cols = custom #Allows users to use a custom set of columns with the model
    elif minimal: #Without private medical data
        cols = ['const', 'GDP_per_capita', 'Schooling', 'region_Asia', 'region_Central America and Caribbean', 'region_European Union','region_Middle East', 'region_North America', 
                'region_Oceania','region_Rest of Europe', 'region_South America']

    else: #With private medical data
        cols = ['const','Adult_mortality','Under_five_deaths',
                'GDP_per_capita','region_Central America and Caribbean', 'region_European Union']
        
    #Creating and fitting the model
    lin_reg = sm.OLS(y_train, X_train_fe[cols])
    results = lin_reg.fit()

    #Using model to make predictions on training data and report Root Mean Squared Error
    y_pred = results.predict(X_train_fe[cols])
    train_rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred)
    print(f"Training data RMSE:\t{train_rmse}")

    #Feature Engineering on test data to match training
    X_test_fe = feature_engineering(X_test)

    #Predicting on testing data and reporting RMSE
    y_test_pred = results.predict(X_test_fe[cols])
    test_rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)
    print(f"Testing data RMSE:\t{test_rmse}")

    return results


In [27]:
results = linear_regression("Life Expectancy Data.csv", minimal=False) 
results.summary()

Training data RMSE:	1.4253607687371903
Testing data RMSE:	1.402238000966035


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.952e+04
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        13:27:07   Log-Likelihood:                -4062.8
No. Observations:                2291   AIC:                             8138.
Df Residuals:                    2285   BIC:                             8172.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   80.3713      0.084    961.959      0.000      80.207      80.535
Adult_mortality                         -0.0447      0.000    -99.559      0.000      -0.046      -0.044
Under_five_deaths                       -0.0884      0.001    -77.075      0.000      -0.091      -0.086
GDP_per_capita                           0.0506      0.002     23.415      0.000       0.046       0.055
region_Central America and Caribbean     1.2566      0.101     12.480      0.000       1.059       1.454
region_European Union                    1.0767      0.095     11.309      0.000       0.890       1.263
==============================================================================
Omnibus:                       38.047   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.918
Skew:                          -0.041   Prob(JB):                     2.42e-16
Kurtosis:                       3.864   Cond. No.                         871.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
list(results.params)

[53.29586723292625,
 0.12243668415386055,
 0.928052406587341,
 8.456219692528556,
 10.938777802504932,
 10.662916648072365,
 11.117855051281126,
 10.434409816790037,
 7.733166882339386,
 8.775122941806782,
 11.260807581924542]

In [20]:
vif_cols = ['Under_five_deaths', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'alcohol_exp', 'const']

cols_stepwise = ['Adult_mortality', 'Economy_status_Developed', 'region_Central America and Caribbean', 'region_South America', 'Under_five_deaths', 'GDP_per_capita', 'region_Oceania','region_European Union', 'Schooling', 'BMI', 'Year', 'Incidents_HIV', 'Hepatitis_B', 'const']

cols_stepwise_vif = ['region_Central America and Caribbean', 'region_South America',
       'Under_five_deaths', 'GDP_per_capita', 'region_Oceania',
       'region_European Union', 'Schooling', 'Incidents_HIV', 'const']

cols = ['const', 'Year', 'Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America', 'gdp_per_capita_exp',
       'alcohol_exp']

cols_ethical = ['const', 'Year', 'Alcohol_consumption', 'GDP_per_capita',
       'Population_mln', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing', 'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America']

cols_ethical_stepwise = ['Schooling', 'const', 'GDP_per_capita', 'region_Central America and Caribbean', 'region_Middle East', 'region_South America', 'region_Asia', 'Economy_status_Developed', 'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania', 'Year', 'region_European Union', 'region_North America', 'Alcohol_consumption', 'Population_mln']

cols_ethical_vif = ['GDP_per_capita', 'Population_mln', 'Economy_status_Developing',
       'region_Asia', 'region_Central America and Caribbean',
       'region_European Union', 'region_Middle East', 'region_North America',
       'region_Oceania', 'region_Rest of Europe', 'region_South America', 'const']

cols_ethical_stepwise_vif = ['GDP_per_capita', 'region_Central America and Caribbean',
       'region_Middle East', 'region_South America', 'region_Asia',
       'Economy_status_Developing', 'region_Rest of Europe', 'region_Oceania',
       'region_European Union', 'region_North America', 'Population_mln']

cols_sumye = ['const',
       'Under_five_deaths', 
       'Adult_mortality',
       'GDP_per_capita',
        'Population_mln'
               ]

cols_hanaa = ['const','Adult_mortality','Under_five_deaths',
       'GDP_per_capita','region_Central America and Caribbean', 'region_European Union']

cols_ethical_sumye = ['const',
       'GDP_per_capita',
       'Schooling',
       'region_Asia',
       'region_Central America and Caribbean', 'region_European Union',
       'region_Middle East', 'region_North America', 'region_Oceania',
       'region_Rest of Europe', 'region_South America'
               ]